Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 09: MolCT with ACT

In [1]:
import time
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore import Tensor
from mindspore import context
from mindspore import dataset as ds
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from cybertron import Cybertron
from cybertron import MolCT
from cybertron import AtomwiseReadout
from cybertron.train import MAE, RMSE, MLoss, MSELoss
from cybertron.train import WithForceLossCell, WithForceEvalCell
from cybertron.train import TrainMonitor
from cybertron.train import TransformerLR

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
sys_name = 'dataset_ethanol_normed_'

train_file = sys_name + 'trainset_1024.npz'
valid_file = sys_name + 'validset_128.npz'

train_data = np.load(train_file)
valid_data = np.load(valid_file)

atom_types = Tensor(train_data['Z'], ms.int32)
scale = train_data['scale']
shift = train_data['shift']

In [3]:
mod = MolCT(
    cutoff=1,
    n_interaction=3,
    dim_feature=128,
    n_heads=8,
    max_cycles=10,
    length_unit='nm',
)

In [4]:
readout = AtomwiseReadout(mod, dim_output=1)

In [5]:
net = Cybertron(mod, readout=readout, atom_types=atom_types, length_unit='nm')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
    Using fixed atom type index:
       Atom 0:     6
       Atom 1:     6
       Atom 2:     8
       Atom 3:     1
       Atom 4:     1
       Atom 5:     1
       Atom 6:     1
       Atom 7:     1
       Atom 8:     1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of ba

In [6]:
tot_params = 0
for i, param in enumerate(net.trainable_params()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 128)
1 model.dis_filter.linear.weight (128, 64)
2 model.dis_filter.linear.bias (128,)
3 model.dis_filter.residual.nonlinear.mlp.0.weight (128, 128)
4 model.dis_filter.residual.nonlinear.mlp.0.bias (128,)
5 model.dis_filter.residual.nonlinear.mlp.1.weight (128, 128)
6 model.dis_filter.residual.nonlinear.mlp.1.bias (128,)
7 model.interactions.0.positional_embedding.norm.gamma (128,)
8 model.interactions.0.positional_embedding.norm.beta (128,)
9 model.interactions.0.positional_embedding.x2q.weight (128, 128)
10 model.interactions.0.positional_embedding.x2k.weight (128, 128)
11 model.interactions.0.positional_embedding.x2v.weight (128, 128)
12 model.interactions.0.multi_head_attention.output.weight (128, 128)
13 model.interactions.0.pondering.dense.weight (1, 384)
14 model.interactions.0.pondering.dense.bias (1,)
15 model.interactions.1.positional_embedding.norm.gamma (128,)
16 model.interactions.1.positional_embedding.norm.beta (128,)
17 model.i

In [7]:
N_EPOCH = 8
REPEAT_TIME = 1
BATCH_SIZE = 32

In [8]:
ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'F': train_data['F'], 'E': train_data['E']}, shuffle=True)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.repeat(REPEAT_TIME)

In [9]:
ds_valid = ds.NumpySlicesDataset(
    {'R': valid_data['R'], 'F': valid_data['F'], 'E': valid_data['E']}, shuffle=False)
ds_valid = ds_valid.batch(128)
ds_valid = ds_valid.repeat(1)

In [10]:
force_dis = train_data['avg_force_dis']
loss_fn = MSELoss(ratio_energy=1, ratio_forces=100, force_dis=force_dis)
loss_network = WithForceLossCell('RFE', net, loss_fn)
eval_network = WithForceEvalCell('RFE', net, loss_fn, scale=scale, shift=shift)

WithForceLossCell with input type: RFE
WithForceEvalCell with input type: RFE
   with scaleshift for training and evaluate dataset:
   Scale: [1.9488194]
   Shift: [-45185.31]
   Scaleshift mode: atomwise


In [11]:
lr = TransformerLR(learning_rate=1., warmup_steps=4000, dimension=128)
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [12]:
energy_mae = 'EnergyMAE'
forces_mae = 'ForcesMAE'
forces_rmse = 'ForcesRMSE'
eval_loss = 'EvalLoss'
model = Model(loss_network, eval_network=eval_network, optimizer=optim,
              metrics={eval_loss: MLoss(), energy_mae: MAE([1, 2]), forces_mae: MAE([3, 4]),
                       forces_rmse: RMSE([3, 4], atom_aggregate='sum')})

In [13]:
outdir = 'Tutorial_C09'
outname = outdir + '_' + net.model_name
record_cb = TrainMonitor(model, outname, per_epoch=1, avg_steps=32,
                         directory=outdir, eval_dataset=ds_valid, best_ckpt_metrics=forces_rmse)


In [14]:

config_ck = CheckpointConfig(save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(prefix=outname, directory=outdir, config=config_ck)

In [15]:
print("Start training ...")
beg_time = time.time()
model.train(N_EPOCH, ds_train, callbacks=[
    record_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

[WARNING] ME(890703:139873186428160,MainProcess):2023-03-29-10:53:30.708.351 [mindspore/train/model.py:1077] For TrainMonitor callback, {'begin', 'step_end', 'epoch_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.


Start training ...
Epoch: 1, Step: 32, Learning_rate: 1.0830951e-05, Last_Loss: 1.3929353, Avg_loss: 1.3492758758366108, EvalLoss: 1.3579516410827637, EnergyMAE: 14.694091796875, ForcesMAE: 814.9491102430555, ForcesRMSE: 1902.6349565215546
Epoch: 2, Step: 64, Learning_rate: 2.2011289e-05, Last_Loss: 1.3086177, Avg_loss: 1.3244402967393398, EvalLoss: 1.30900239944458, EnergyMAE: 14.380126953125, ForcesMAE: 799.9830729166666, ForcesRMSE: 1867.9343314652865
Epoch: 3, Step: 96, Learning_rate: 3.3191627e-05, Last_Loss: 0.9497292, Avg_loss: 1.2303790003061295, EvalLoss: 1.1438555717468262, EnergyMAE: 13.700439453125, ForcesMAE: 743.26171875, ForcesRMSE: 1745.7551056459463
Epoch: 4, Step: 128, Learning_rate: 4.437197e-05, Last_Loss: 0.6313386, Avg_loss: 0.9160487297922373, EvalLoss: 0.7151620388031006, EnergyMAE: 12.183349609375, ForcesMAE: 553.6905381944445, ForcesRMSE: 1378.417450073332
Epoch: 5, Step: 160, Learning_rate: 5.5552304e-05, Last_Loss: 0.5404496, Avg_loss: 0.5693284701555967, Ev